Remaining items:
- docstring hunt for patch rulesnd citations
- plot patches "overflowing" beyond the inner tract region
- plot dp1 tract boundary
- add lsdb and dask to doc deps (hmm...is it worth it? tho the plot is pretty clutch)

# Skymaps Overview

In this notebook, we will discuss:
- Ring Skymaps (the lsst.skymap class to which our original LSST skymap belongs)
- Some basic properties of ring skymaps
- The definitions and differences of tracts (inner region), tracts (outer polygons), and patches

In [ ]:
from skymap_convert._internal.examples import demo_rings_plot, demo_rings_tracts_plot

## The anatomy of a ring skymap

In a ring skymap, we divide the sky (which is unfortunately spherical) into horizontal rings.

The LSST skymap has over 200 rings. For illustrative purposes, let us imagine a skymap with 5 rings:

In [ ]:
demo_rings_plot()

Then, we divide each of those things into tracts (inner region).

In [ ]:
demo_rings_tracts_plot()

Some basic properties of tracts (inner region) are:
- They have the same area and dimensions as the other patches. <a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1)
- There are no gaps or overlaps between all the patches in a tract. <a name="cite_ref-2"></a>[<sup>[2]</sup>](#cite_note-2)
- Patches *do* need spherical geometry/knowledge of the WCS to calculate. So, these are the reason we've made our converted skymaps be a vertex dump, rather than the lighter-weight legacy RingOptimized version.

<a name="cite_note-1"></a>1. [^](#cite_ref-1) TODO grab the docstring that describes this, if possible  
<a name="cite_note-2"></a>2. [^](#cite_ref-2) TODO grab the docstring that describes this, if possible 

Note that patches can and do "overflow" beyond the boundaries of the tract (inner region).

TODO : if possible, plot patches and a tract inner region.

 If we draw a boundary based on the outside boundary of all the patches in a tract, we get the bounds of the tract (outer polygon).  

 However, tract (outer polygon) seems to be pretty useless to know—but we mention it so we don't get it confused with tract (inner region).

## Applied to DP1 tract classifications

 Based on a quick sketch with DP1 data, the tract that an object is assigned to appears to be based on the tract (inner region).

*ASK NEVEN: This is ok, right, if we make sure to only show ra/dec (and no prints) of a limited number of points?*

In [ ]:
from skymap_convert import ConvertedSkymapReader
from pathlib import Path

package_root = Path.home() / "code" / "skymap-convert"

converted_skymap_path = package_root / "converted_skymaps" / "lsst_skymap"
converted_skymap_path

reader = ConvertedSkymapReader(converted_skymap_path)

In [ ]:
import lsdb

In [ ]:
from dask.distributed import Client

client = Client(n_workers=4, memory_limit="auto")
client

In [ ]:
dp1_object_collection = lsdb.open_catalog(
    "/sdf/data/rubin/shared/lsdb_commissioning/hats/v29_0_0/dia_object_collection",
    columns=["dec", "ra", "tract"],
)
dp1_object_box = dp1_object_collection.box_search(ra=[37.75, 40.0], dec=[6.0, 7.75])

In [ ]:
dp1_computed = dp1_object_box.compute()

In [ ]:
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates import SkyCoord


def _plot_patches(ax, tract_id, patch_ids, single_color=None, alpha=0.5, marker_size=5):
    """Plot the patches on the given axes."""
    for patch_id in patch_ids:
        patch_verts = reader.get_patch_vertices(tract_id, patch_id)
        if patch_verts is None:
            raise ValueError(f"Patch {patch_id} not found in tract {tract_id}.")
        ra, dec = zip(*patch_verts, strict=True)
        skycoord = SkyCoord(ra=ra * u.degree, dec=dec * u.degree, frame="icrs")
        ra_deg = skycoord.ra.wrap_at(360 * u.deg).deg
        dec_deg = skycoord.dec.deg

        # Close the polygon
        ra_deg = np.append(ra_deg, ra_deg[0])
        dec_deg = np.append(dec_deg, dec_deg[0])

        # Plot the patches with a single color if specified
        if single_color is not None:
            ax.plot(
                ra_deg,
                dec_deg,
                color=single_color,
                marker="o",
                linestyle="-",
                alpha=alpha,
                markersize=marker_size,
            )
        else:
            ax.plot(ra_deg, dec_deg, marker="o", linestyle="-", alpha=alpha, markersize=marker_size)


def _plot_tract(ax, tract_id, color, inner=True):
    """Plot the tract boundary for a given tract ID."""
    # tract_radec_verts = get_orig_tract_radec_verts(tract_id, inner=inner)
    tract_radec_verts = reader.get_tract_vertices(tract_id)
    ra, dec = zip(*tract_radec_verts, strict=True)
    skycoord = SkyCoord(ra=ra * u.degree, dec=dec * u.degree, frame="icrs")
    ra_deg = skycoord.ra.wrap_at(360 * u.deg).deg
    dec_deg = skycoord.dec.deg

    # Close the polygon
    ra_deg = np.append(ra_deg, ra_deg[0])
    dec_deg = np.append(dec_deg, dec_deg[0])

    # Plot the tract boundary
    if inner:
        # Plot with a solid color fill
        ax.fill(ra_deg, dec_deg, color=color, alpha=0.25, label=f"Tract {tract_id} Inner")
    else:
        ax.plot(ra_deg, dec_deg, color=color, linewidth=3, alpha=0.8, label=f"Tract {tract_id} Outer")


def _get_ra_dec_range(patches):
    """Get the RA/Dec range from the patches."""
    min_ra, max_ra = float("inf"), float("-inf")
    min_dec, max_dec = float("inf"), float("-inf")
    for patch in patches:
        ra, dec = zip(*patch, strict=True)
        min_ra = min(min(ra), min_ra)
        max_ra = max(max(ra), max_ra)
        min_dec = min(min(dec), min_dec)
        max_dec = max(max(dec), max_dec)
    return (min_ra, max_ra, min_dec, max_dec)

def _plot_data(ax, data=None, target_tract=None):
    """Plot the data points on the given axes."""
    if data is not None and target_tract is not None:
        for i in range(len(data)):
            row = data.iloc[i]
            ra, dec = row["ra"], row["dec"]
            color = "blue" if row["tract"] == target_tract else "red"
            ax.plot(ra, dec, marker=".", color=color, markersize=5, alpha=0.7)
    else:
        print("No data to plot, or target tract has not been specified.")

In [ ]:
def plot_patches_in_tract(tract_id, data=None):
    """Plot the patches in a specific tract with zoomed-in view."""
    fig, ax = plt.subplots(figsize=(10, 6))

    # Plot the patches and tract boundaries
    patch_ids = range(100)
    # _plot_patches(ax, patches, single_color="blue")
    _plot_tract(ax, tract_id, inner=True, color="black")
    _plot_tract(ax, tract_id, inner=False, color="black")

    # Plot the data, if provided
    _plot_data(ax, data, target_tract=tract_id)

    # Set zoom level based on the RA/Dec range of the patches
    min_ra, max_ra, min_dec, max_dec = _get_ra_dec_range(patches)
    ax.set_xlim(min_ra - 1, max_ra + 1)
    ax.set_ylim(min_dec - 0.25, max_dec + 0.25)

    # Add a legend for the tract boundaries, labels, and title
    ax.legend(loc="upper right", fontsize="small")
    ax.set_xlabel("RA (deg)")
    ax.set_ylabel("Dec (deg)")
    ax.set_title(f"Zoomed-In View of DP1 Subset Tract Designation for Tract {tract_id}")
    ax.grid(True)

    ax.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
plot_patches_in_tract(10464, dp1_computed)

Since tracts (inner region) are designed such that they cover the entire (~~stupidly spherical~~) sky with no gaps or overlaps, we can infer the existence of a surjective function mapping from the set of all possible radec sphere coordinates to the set of tracts in our skymap.

That is to say, if we have a radec coordinate, we have exactly one tract to which it corresponds. Neat!

## About  

**Author:** Olivia Lynn

**Last updated on:** July 11, 2025